# Creating the Training Dataset

In [1]:
import tensorflow as tf

2025-03-19 06:40:17.708943: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 06:40:17.843555: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742345717.898156     269 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742345717.913961     269 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 06:40:18.043838: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
shakespeare_url = 'https://homl.info/shakespeare' 
filepath = tf.keras.utils.get_file('shakespear.txt',shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [3]:
print(shakespeare_text[:88])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First 


In [4]:
text_vec_layer = tf.keras.layers.TextVectorization(split='character',standardize='lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
print(encoded)

2025-03-19 06:40:22.089437: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


tf.Tensor([21  7 10 ... 22 28 12], shape=(1115394,), dtype=int64)


In [5]:
encoded -=2 #drop tokens 0 (pad) and 1(unknown), which we will not use
n_tokens = text_vec_layer.vocabulary_size() - 2 #number of distictc chars = 39
dataset_size = len(encoded) #total number of chars = 
print(dataset_size)

1115394


In [6]:
# a utility function to shift window length  + 1, and remove  the last char in a sequecne and make a next char a target char
def to_dataset(sequence,length, shuffle = False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length +1 ))
    if shuffle:
        ds = ds.shuffle(buffer_size = 100_000,seed = seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window:(window[:,:-1],window[:,1:])).prefetch(1)


In [7]:
# creating train,validation and test set Now !
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length,shuffle =True, seed = 42)

valid_set = to_dataset(encoded[1_000_000:1_060_000],length= length)
test_set = to_dataset(encoded[1_060_000:],length=length)

# Building and Training the Char-RNN Model

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim = 16),
    tf.keras.layers.GRU(128,return_sequences=True),
    tf.keras.layers.Dense(n_tokens,activation='softmax')
    ])

model.compile(loss = 'sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])

model_ckpt = tf.keras.callbacks.ModelCheckpoint('my_shakespeare_model.keras',monitor='val_accuracy',save_best_only= True)

history = model.fit(train_set,validation_data=valid_set,epochs = 5,callbacks = [model_ckpt],batch_size= 1000)

Epoch 1/5
  31247/Unknown 1417s 45ms/step - accuracy: 0.5434 - loss: 1.5120

2025-03-18 17:16:57.492638: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1442s 46ms/step - accuracy: 0.5434 - loss: 1.5120 - val_accuracy: 0.5298 - val_loss: 1.6178
Epoch 2/5


2025-03-18 17:17:22.724448: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1611s 51ms/step - accuracy: 0.5927 - loss: 1.3095 - val_accuracy: 0.5349 - val_loss: 1.5992
Epoch 3/5


2025-03-18 17:44:13.904838: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1402s 45ms/step - accuracy: 0.5966 - loss: 1.2909 - val_accuracy: 0.5371 - val_loss: 1.5880
Epoch 4/5
31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1470s 47ms/step - accuracy: 0.5985 - loss: 1.2815 - val_accuracy: 0.5389 - val_loss: 1.5754
Epoch 5/5


2025-03-18 18:32:05.955354: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


31247/31247 ━━━━━━━━━━━━━━━━━━━━ 1359s 43ms/step - accuracy: 0.6002 - loss: 1.2743 - val_accuracy: 0.5389 - val_loss: 1.5776


### Wrapping the model to preprocess text 

In [79]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X:X -2),
    loaded_model
])

In [78]:
def token_text(shakespeare_text):
    text_vec_layer = tf.keras.layers.TextVectorization(split='character',standardize='lower')
    text_vec_layer.adapt([shakespeare_text])
    return text_vec_layer


In [82]:
import numpy as np
input_text ='To be or not to b'

y_proba = shakespeare_model.predict(input_text)[0,-1]
y_pred = tf.argmax(y_proba) # choose the most probable character ID
predicted_char = text_vec_layer.get_vocabulary()[y_pred + 2]
print(predicted_char)

ValueError: Unrecognized data type: x=To be or not to b (of type <class 'str'>)

In [8]:
#model.save('my_shakespeare_model.keras')
loaded_model = tf.keras.models.load_model("my_shakespeare_model.keras")
loaded_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ (None, None, 16)       │           624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_2 (GRU)                     │ (None, None, 128)      │        56,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 39)       │         5,031 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 185,160 (723.29 KB)

 Trainable params: 61,719 (241.09 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 123,441 (482.20 KB)

In [15]:
# Evaluate the restored model
loss, acc = loaded_model.evaluate(test_set, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))


1728/1728 - 25s - 15ms/step - accuracy: 0.5076 - loss: 1.7464
Restored model, accuracy: 50.76%


2025-03-19 06:31:48.462849: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/home/harry/Documents/Code/Data_Science/DeepLearning/.venv_tf/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


# Generating fake shakespearean text

In [83]:
log_probas = tf.math.log([[0.5, 0.4, 0.1]])  # probas = 50%, 40%, and 10%
tf.random.set_seed(42)
tf.random.categorical(log_probas, num_samples=8)


<tf.Tensor: shape=(1, 8), dtype=int64, numpy=array([[0, 1, 0, 2, 1, 0, 0, 1]])>

In [84]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=0.01))


ValueError: Unrecognized data type: x=['To be or not to be'] (of type <class 'list'>)

In [85]:
print(extend_text("To be or not to be", temperature=1))
print(extend_text("To be or not to be", temperature=100))


ValueError: Unrecognized data type: x=['To be or not to be'] (of type <class 'list'>)

# Stateful RNN

In [91]:
encoded.shape

TensorShape([1115394])

In [92]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1 ,shift = length,drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length +1)).batch(1)
    return ds.map(lambda window : (window[:,:-1],window[:,1:])).prefetch(1)



In [94]:
stateful_train_set = to_dataset_for_stateful_rnn(encoded[1_000_000:],length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000],length)

stateful_test_set =  to_dataset_for_stateful_rnn(encoded[1_060_000:],length)

In [114]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

ValueError: Unrecognized keyword arguments passed to Embedding: {'batch_input_shape': [1, None]}

In [116]:
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()


model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set,
                    epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])


NameError: name 'model' is not defined